# Voyage AI Quick Start

This notebook is a companion to the [Voyage AI Quick Start](https://www.mongodb.com/docs/voyageai/quickstart/) tutorial. Refer to the page for set-up instructions and detailed explanations.

In this guide, you learn how to generate your first vector embeddings with Voyage AI and build a basic application.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/voyageai/notebooks/quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Create a Model API Key

To access Voyage AI models, create a Model API Key in the MongoDB Atlas UI.

1. [Sign up for a free Atlas account or log in.](https://www.mongodb.com/cloud/atlas/register)
2. Create a model API Key for your project:
   - In your Atlas project, select **AI Models** from the navigation bar.
   - Click **Create Model API Key**.
   - Give the API key a name and then click **Create**.
3. Set the API key in your environment:
   - Copy the API key and store it in a safe location.
   - Export the API key as an environment variable.

## Install Required Packages

In [ ]:
!pip install --upgrade voyageai numpy

## Set Environment Variables

In [ ]:
import os

# Set your API key here or export it as an environment variable
os.environ["VOYAGE_API_KEY"] = "<your-model-api-key>"

## Generate Your First Embeddings

In this section, you generate vector embeddings using a Voyage AI embedding model and the Python client.

In [ ]:
import voyageai

# Initialize Voyage client
vo = voyageai.Client()

# Sample texts
texts = [
    "hello, world",
    "welcome to voyage ai!"
]

# Generate embeddings
result = vo.embed(
    texts,
    model="voyage-4-large"
)

print(f"Generated {len(result.embeddings)} embeddings")
print(f"Each embedding has {len(result.embeddings[0])} dimensions")
print(f"First embedding (truncated): {result.embeddings[0][:5]}...")

## Build a Basic RAG Application

Now that you know how to generate vector embeddings, build a basic RAG application to learn how to use Voyage AI models to implement AI search and retrieval. RAG enables LLMs to generate context-aware responses by retrieving relevant information from your data before generating answers.

> **Note:** RAG applications require access to an LLM. This tutorial provides examples using Anthropic or OpenAI, but you can use any LLM provider of your choice.

In [ ]:
import voyageai
import numpy as np

# Initialize Voyage client
vo = voyageai.Client()

# Sample documents
documents = [
    "This quarter, our company is focused on building new products, increasing market share, and cutting costs.",
    "20th-century innovations, from radios to smartphones, centered on electronic advancements.",
    "Photosynthesis in plants converts light energy into glucose and produces essential oxygen."
]

query = "What are my company's goals this quarter?"

### Generate Embeddings and Perform Semantic Search

In [ ]:
# Generate embeddings for documents
doc_embeddings = vo.embed(
    texts=documents,
    model="voyage-4-large",
    input_type="document"
).embeddings

# Generate embedding for query
query_embedding = vo.embed(
    texts=[query],
    model="voyage-4-large",
    input_type="query"
).embeddings[0]

# Calculate similarity scores using dot product
similarities = np.dot(doc_embeddings, query_embedding)

# Sort by similarity (np.argsort with negative sign sorts high to low)
ranked_indices = np.argsort(-similarities)

print(f"Semantic search result: {documents[ranked_indices[0]][:50]}...")
print(f"Similarity score: {similarities[ranked_indices[0]]:.4f}\n")

### Refine Results with Reranking

In [ ]:
# Refine results with reranking model
reranked = vo.rerank(query, documents, model="rerank-2.5", top_k=3)

print("Reranked results:")
for i, result in enumerate(reranked.results, 1):
    print(f"{i}. Score: {result.relevance_score:.4f} - {result.document[:50]}...")

# Get the most relevant document as context
context = reranked.results[0].document

### Generate Response with LLM

In [ ]:
!pip install anthropic openai

In [ ]:
# Choose your LLM provider and set the appropriate API key
LLM_PROVIDER = "anthropic"  # Options: "anthropic" or "openai"
os.environ["ANTHROPIC_API_KEY"] = "<your-anthropic-api-key>"
os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# Initialize LLM client based on provider
if LLM_PROVIDER == "anthropic":
    import anthropic
    llm_client = anthropic.Anthropic()
elif LLM_PROVIDER == "openai":
    from openai import OpenAI
    llm_client = OpenAI()
else:
    raise ValueError("Choose a different LLM provider: 'anthropic' or 'openai'.")

In [ ]:
# Generate answer using retrieved context
prompt = f"Based on this information: '{context}', answer: {query}"

if LLM_PROVIDER == "anthropic":
    response = llm_client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=1024,
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.content[0].text
elif LLM_PROVIDER == "openai":
    response = llm_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    answer = response.choices[0].message.content
else:
    answer = "Configure another LLM provider."

print(f"\nQuestion: {query}")
print(f"Answer: {answer}")